# Basic Azure Resource Management

## PowerShell

PowerShell is a powerful scripting and automation framework developed by Microsoft. It is designed for task automation and configuration management and is particularly useful for managing and automating Microsoft Windows environments. PowerShell uses a command-line interface with a scriptable approach, and it's built on the .NET Framework.

* [What is PowerShell?](https://learn.microsoft.com/powershell/scripting/overview?view=powershell-7.3&WT.mc_id=AZ-MVP-5004796)

## PowerShell and Microsoft Azure

When it comes to Microsoft Azure, PowerShell provides a robust set of cmdlets (pronounced "command-lets") that enable you to interact with and manage Azure resources, making it a valuable tool for working with Azure services. 

When you run a PowerShell cmdlet to, for example, create a virtual machine or retrieve information about an Azure resource, the cmdlet translates your request into an HTTP request to the relevant Azure REST API endpoint.

* [Azure PowerShell Documentation](https://learn.microsoft.com/powershell/azure/?view=azps-10.4.1&WT.mc_id=AZ-MVP-5004796)

## Resource Creation

In [2]:
# Import Module
Import-Module Az -Verbose

VERBOSE: Loading module from path '/root/.local/share/powershell/Modules/Az/10.4.1/Az.psd1'.


In [ ]:
#Create Azure Resource Group
New-AzResourceGroup -Name "MyResourceGroup" -Location "West US"

In [15]:
# Get Regions

Get-AzLocation | Select-Object -First 1
Get-AzLocation | Select-Object DisplayName, Location, PhysicalLocation, GeographyGroup | Format-Table



Location         : eastus
DisplayName      : East US
Type             : Region
Longitude        : -79.8164
Latitude         : 37.3719
PhysicalLocation : Virginia
RegionType       : Physical
RegionCategory   : Recommended
GeographyGroup   : US
PairedRegion     : [
                     {
                       "Name": "westus",
                       "Id": 
                   "/subscriptions/6bca53bc-98d9-4cd3-92e7-0364c7bffac4/locations/westus",
                       "SubscriptionId": null
                     }
                   ]
Providers        : {Microsoft.ContainerRegistry, Microsoft.AlertsManagement, microsoft.insights, 
                   Microsoft.DesktopVirtualization…}



Error: Microsoft.DotNet.Interactive.NoSuitableKernelException: No kernel found for Microsoft.DotNet.Interactive.Commands.RequestHoverText with target kernel 'pwsh'.
   at Microsoft.DotNet.Interactive.Commands.KernelCommand.InvokeAsync(KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Commands\KernelCommand.cs:line 184
   at Microsoft.DotNet.Interactive.Kernel.HandleAsync(KernelCommand command, KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.cs:line 324
   at Microsoft.DotNet.Interactive.KernelCommandPipeline.<BuildPipeline>b__6_0(KernelCommand command, KernelInvocationContext context, KernelPipelineContinuation _) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\KernelCommandPipeline.cs:line 60
   at Microsoft.DotNet.Interactive.KernelCommandPipeline.SendAsync(KernelCommand command, KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\KernelCommandPipeline.cs:line 51

Error: Microsoft.DotNet.Interactive.NoSuitableKernelException: No kernel found for Microsoft.DotNet.Interactive.Commands.RequestHoverText with target kernel 'pwsh'.
   at Microsoft.DotNet.Interactive.Commands.KernelCommand.InvokeAsync(KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Commands\KernelCommand.cs:line 184
   at Microsoft.DotNet.Interactive.Kernel.HandleAsync(KernelCommand command, KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.cs:line 324
   at Microsoft.DotNet.Interactive.KernelCommandPipeline.<BuildPipeline>b__6_0(KernelCommand command, KernelInvocationContext context, KernelPipelineContinuation _) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\KernelCommandPipeline.cs:line 60
   at Microsoft.DotNet.Interactive.KernelCommandPipeline.SendAsync(KernelCommand command, KernelInvocationContext context) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\KernelCommandPipeline.cs:line 51



DisplayName          Location           PhysicalLocation GeographyGroup
-----------          --------           ---------------- --------------
East US              eastus             Virginia         US
East US 2            eastus2            Virginia         US
South Central US     southcentralus     Texas            US
West US 2            westus2            Washington       US
West US 3            westus3            Phoenix          US
Australia East       australiaeast      New South Wales  Asia Pacific
Southeast Asia       southeastasia      Singapore        Asia Pacific
North Europe         northeurope        Ireland          Europe
Sweden Central       swedencentral      Gävle            Europe
UK South             uksouth            London           Europe
West Europe          westeurope         Netherlands      Europe
Central US           centralus          Iowa             US
South Africa North   southafricanorth   Johannesburg     Africa
Central India        centralindia 

In [19]:
#Create Azure Resource Group
$region = 'AustraliaEast'
New-AzResourceGroup -Name "MyResourceGroup$region" -Location $region


ResourceGroupName : MyResourceGroupAustraliaEast
Location          : australiaeast
ProvisioningState : Succeeded
Tags              : 
ResourceId        : /subscriptions/6bca53bc-98d9-4cd3-92e7-0364c7bffac4/resourceGroups/MyResourceGr
                    oupAustraliaEast




In [29]:
# Create a storage account (attempt1)

$region = 'AustraliaEast'
$ResourceGroupName = "MyResourceGroup$region"
New-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name "mystorageaccount" -Location $region -SkuName Standard_LRS -AllowBlobPublicAccess $false

Default value of AllowBlobPublicAccess and AllowCrossTenantReplication settings on storage account will be changed to False in the future release. 
When AllowBlobPublicAccess is False on a storage account, container ACLs cannot be configured to allow anonymous access to blobs within the storage account. 
When AllowCrossTenantReplication is False on a storage account, cross AAD tenant object replication is not allowed when setting up Object Replication policies.
- The change is expected to take effect in Az version : '11.0.0'
- The change is expected to take effect in Az.Storage version : '6.0.0'
Cmdlet invocation changes :
    Old Way : AllowBlobPublicAccess and AllowCrossTenantReplication are set to True by defult.
    New Way : AllowBlobPublicAccess and AllowCrossTenantReplication are set to False by default.
Note : Go to https://aka.ms/azps-changewarnings for steps to suppress this breaking change warning, and other information on breaking changes in Azure PowerShell.
New-AzStorageA

Error: Command failed: SubmitCode: # Create a storage account (attempt1) ...

In [44]:
# Create a storage account (attempt2)
$uniqueId = [guid]::NewGuid().ToString().Substring(0,4)
$region = 'AustraliaEast'
$ResourceGroupName = "MyResourceGroup$region"
New-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name "mystgacc$uniqueId" -Location $region -SkuName Standard_LRS -AllowBlobPublicAccess $false -verbose

Default value of AllowBlobPublicAccess and AllowCrossTenantReplication settings on storage account will be changed to False in the future release. 
When AllowBlobPublicAccess is False on a storage account, container ACLs cannot be configured to allow anonymous access to blobs within the storage account. 
When AllowCrossTenantReplication is False on a storage account, cross AAD tenant object replication is not allowed when setting up Object Replication policies.
- The change is expected to take effect in Az version : '11.0.0'
- The change is expected to take effect in Az.Storage version : '6.0.0'
Cmdlet invocation changes :
    Old Way : AllowBlobPublicAccess and AllowCrossTenantReplication are set to True by defult.
    New Way : AllowBlobPublicAccess and AllowCrossTenantReplication are set to False by default.
Note : Go to https://aka.ms/azps-changewarnings for steps to suppress this breaking change warning, and other information on breaking changes in Azure PowerShell.

StorageAccoun

In [45]:
#Remove Azure Storage account
$region = 'AustraliaEast'
$ResourceGroupName = $ResourceGroupName
Remove-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name "mystgacc$uniqueId" -Force -verbose
Get-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name "mystgacc$uniqueId" -verbose

VERBOSE: Performing the operation "Remove Storage Account" on target "mystgacce3b9".
Get-AzStorageAccount: 
Line |
   5 |  Get-AzStorageAccount -ResourceGroupName $ResourceGroupName -Name "mys …
     |  ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
     | The Resource 'Microsoft.Storage/storageAccounts/mystgacce3b9' under resource group 'MyResourceGroupAustraliaEast' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix


Error: Command failed: SubmitCode: #Remove Azure Storage account ...